#### QSS Rate

In [ ]:
import h5py
import numpy as np
import pandas as pd
import dill as pickle

from matplotlib import pyplot as plt

Define paths

In [ ]:
T = 1e4
lev_file = "/home/zanardi/Codes/ML/RONEK/ronek/examples/RVC_O3/database/O2.csv"
kin_file = "/home/zanardi/Workspace/AirDatabase/HDF5/QCT/Orig/O3_UMN.hdf5"
sol_file = "/home/zanardi/Codes/ML/RONEK/run/RVC_O3/test/data/case_cold_T10k.p"

Levels

In [ ]:
levels = pd.read_csv(lev_file)
gi = levels["g"].values
ei = levels["E"].values
ei -= ei.min()

Kinetics

In [ ]:
def arrhenius(T, A, beta, Ta):
  return A * np.exp(beta*np.log(T) - Ta/T)

In [ ]:
rates = h5py.File(kin_file, "a")
param = [rates[f"Arrhenius/Diss_Corr/{k}"][()] for k in ("A", "beta", "Ta")]
kd = arrhenius(T, *param)

Solution

In [ ]:
sol = pickle.load(open(sol_file, "rb"))
t = sol["t"]
ni = sol["n"][1]
fi = ni / np.sum(ni, axis=0)

$e_\text{int}$ evolution to determine the QSS region

In [ ]:
e = np.sum(fi.T * ei, axis=-1)

In [ ]:
plt.semilogx(t, e)
plt.xlabel("$t$ [s]")
plt.ylabel("$e$ [eV]")
plt.show()
plt.close()

QSS distribution

In [ ]:
tqss = 3e-6
tidx = np.argmin(np.absolute(t-tqss))
fqss = fi[:,tidx]

In [ ]:
plt.semilogy(
  ei,
  ni[:,tidx]/gi,
  marker="o",
  fillstyle="full",
  markersize=1,
  linestyle="none"
)
plt.xlabel(r"$\epsilon_i$ [eV]")
plt.ylabel(r"$n_i/g_i$ [m$^{-3}$]")
plt.show()
plt.close()

QSS dissociation rate

In [ ]:
kqss = np.sum(fqss * kd)

In [ ]:
text = f"The QSS rate is: {kqss} cm^3/s"
with open("./rate.txt", "w") as file:
  file.write(text)
print(text)